In [1]:
import geodb.model
from geodb.model import GPSPoint, db_url, GPSTrack

In [2]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [3]:
engine = sqlalchemy.create_engine(db_url(), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
import pandas as pd
import numpy as np

Find longest tracks

In [64]:
elapsed_query = """
select track.*, count(*), min(time) as start, max(time) as end, max(time) - min(time) as elapsed
from track
join point on track_id = track.id
group by track.id
order by elapsed desc
"""
display(pd.read_sql_query(elapsed_query, engine).dropna(subset=['elapsed']))

,id,name,comment,description,source,type,filename,properties,raw,count,start,end,elapsed
8,2503,Aneel Nazareth (9962922),None,Aneel Nazareth (9962922),None,None,inreach.gpx,{},None,23,2019-12-09 22:43:30+00:00,2019-12-11 02:46:30+00:00,1 days 04:03:00
9,2500,Aneel Nazareth (9953683),None,Aneel Nazareth (9953683),None,None,inreach.gpx,{},None,2048,2019-12-07 00:03:15+00:00,2019-12-07 23:40:29+00:00,0 days 23:37:14
10,2497,Aneel Nazareth (9947046),None,Aneel Nazareth (9947046),None,None,inreach.gpx,{},None,2111,2019-12-04 23:35:00+00:00,2019-12-05 21:53:17+00:00,0 days 22:18:17
11,2502,Aneel Nazareth (9959973),None,Aneel Nazareth (9959973),None,None,inreach.gpx,{},None,545,2019-12-09 00:15:00+00:00,2019-12-09 21:23:59+00:00,0 days 21:08:59
12,2501,Aneel Nazareth (9956605),None,Aneel Nazareth (9956605),None,None,inreach.gpx,{},None,1785,2019-12-07 23:40:45+00:00,2019-12-08 14:10:00+00:00,0 days 14:29:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,923,None,None,None,None,car,2019-11-28.gpx,{},None,1,2019-11-28 01:51:46+00:00,2019-11-28 01:51:46+00:00,0 days 00:00:00
2500,1867,None,None,None,None,car,2019-12-04.gpx,{},None,1,2019-12-05 02:27:10+00:00,2019-12-05 02:27:10+00:00,0 days 00:00:00
2501,1014,None,None,None,None,car,2019-11-28.gpx,{},None,1,2019-11-28 08:08:10+00:00,2019-11-28 08:08:10+00:00,0 days 00:00:00
2502,520,柚之茶屋,None,None,None,Waypoint,None,None,None,1,2019-11-27 05:12:36+00:00,2019-11-27 05:12:36+00:00,0 days 00:00:00


Pick a track

In [65]:
track2503 = session.query(GPSTrack).get(2503)

In [66]:
track2503.name

'Aneel Nazareth (9962922)'

Find all overlapping tracks

In [73]:
overlapping_query = """
select * from track where id in
(select distinct(track_id) from point
where point.time between (select min(time) from point where track_id = 2503) 
                     and (select max(time) from point where track_id = 2503))
"""
display(pd.read_sql_query(overlapping_query, engine))

,id,name,comment,description,source,type,filename,properties,raw
0,2439,Keisei Ueno Station (KS01) (京成上野駅),None,None,None,Waypoint,None,None,None
1,2440,Keisei Narita Airport Terminal 2-3 Station (KS...,None,None,None,Waypoint,None,None,None
2,2441,Terminal 2 (第2ターミナル),None,None,None,Waypoint,None,None,None
3,2442,ASIAN CAFE Bowl Bowl,None,None,None,Waypoint,None,None,None
4,2443,NRT - GATE 74 (Terminal 2),None,None,None,Waypoint,None,None,None
...,...,...,...,...,...,...,...,...,...
93,2636,Keisei Ueno Station (KS01) (京成上野駅),None,None,None,Waypoint,None,None,None
94,2637,Keisei Narita Airport Terminal 2-3 Station (KS...,None,None,None,Waypoint,None,None,None
95,2638,Terminal 2 (第2ターミナル),None,None,None,Waypoint,None,None,None
96,2639,ASIAN CAFE Bowl Bowl,None,None,None,Waypoint,None,None,None


In [68]:
overlapping = [session.query(GPSTrack).get(int(t)) for t in pd.read_sql_query(overlapping_query, engine)[['id']].values]

In [69]:
len(overlapping)

98

In [74]:
query = """
select track.*, count(*), min(time) as start, max(time) as end, max(time) - min(time) as elapsed
from track
join point on track.id = point.track_id
where point.time between (select min(p2.time) from point p2 where track_id = 2503)
                     and (select max(p3.time) from point p3 where track_id = 2503)
group by track.id
order by elapsed desc
"""
display(pd.read_sql_query(query, engine))

,id,name,comment,description,source,type,filename,properties,raw,count,start,end,elapsed
0,2503,Aneel Nazareth (9962922),None,Aneel Nazareth (9962922),None,None,inreach.gpx,{},None,23,2019-12-09 22:43:30+00:00,2019-12-11 02:46:30+00:00,1 days 04:03:00
1,2509,20191210,None,None,None,None,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None,400,2019-12-10 06:01:17+00:00,2019-12-10 13:07:36+00:00,0 days 07:06:19
2,2508,AAL176-20191210,None,RJAA - KDFW,None,None,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None,111,2019-12-10 02:37:43+00:00,2019-12-10 05:59:52+00:00,0 days 03:22:09
3,2544,None,None,None,None,airplane,2019-12-10-edited.gpx,{},None,27,2019-12-10 16:30:06+00:00,2019-12-10 18:22:01+00:00,0 days 01:51:55
4,2592,None,None,None,None,train,2019-12-09-edited.gpx,{},None,342,2019-12-09 23:00:09+00:00,2019-12-09 23:46:32+00:00,0 days 00:46:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,2481,Austin Bergstrom International Airport (AUS),None,None,None,Waypoint,None,None,None,1,2019-12-10 18:22:16+00:00,2019-12-10 18:22:16+00:00,0 days 00:00:00
94,2604,None,None,None,None,airplane,2019-12-09-edited.gpx,{},None,1,2019-12-10 00:25:32+00:00,2019-12-10 00:25:32+00:00,0 days 00:00:00
95,2480,Gate C35,None,None,None,Waypoint,None,None,None,1,2019-12-10 15:01:42+00:00,2019-12-10 15:01:42+00:00,0 days 00:00:00
96,2479,Gate C35,None,None,None,Waypoint,None,None,None,1,2019-12-10 14:29:44+00:00,2019-12-10 14:29:44+00:00,0 days 00:00:00


In [75]:
from ipyleaflet import (
    Map,
    Marker,
    Polyline, 
    Popup,
    GeoJSON,
    DrawControl
)
from ipywidgets import HTML

from traitlets import link

In [76]:
len(overlapping[0].points)

1

In [77]:
ps = [[float(p.coords[1]), float(p.coords[0])] for p in track2503.points]
bounds = [list(ps[0]), list(ps[0])]
for p in ps:
    if bounds[0][0] > p[0]:
        bounds[0][0] = p[0]
    if bounds[0][1] > p[1]:
        bounds[0][1] = p[1]
    if bounds[1][0] < p[0]:
        bounds[1][0] = p[0]
    if bounds[1][1] < p[1]:
        bounds[1][1] = p[1]
bounds

[[30.20118, -97.713281], [35.783671, 140.38918]]

In [78]:
center = [(bounds[0][0] + bounds[1][0])/2, (bounds[0][1] + bounds[1][1])/2 - 180]

In [79]:
center

[32.992425499999996, -158.6620505]

In [80]:
def add_track(m, t):
    l = track.as_polyline(
        fill=False,
        weight=1)
    m.add_layer(l)
    l.popup = HTML(f"{track.id}: {track.name}")

In [81]:
m = Map(center=center, zoom=2, close_popup_on_click=False)
for track in overlapping:
    add_track(m, track)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [84]:
def clone_model(model, **kwargs):
    """Clone an arbitrary sqlalchemy model object without its primary key values."""
    # Ensure the model’s data is loaded before copying.
    model.id

    table = model.__table__
    non_pk_columns = [k for k in table.columns.keys() if k not in table.primary_key]
    data = {c: getattr(model, c) for c in non_pk_columns}
    data.update(kwargs)

    clone = model.__class__(**data)
    return clone

In [169]:
query = """
select type, count(*) from track
join point on track.id = track_id
group by type
"""
display(pd.read_sql_query(query, engine))

,type,count
0,cycling,79
1,airplane,3061
2,car,11959
3,None,29110
4,stationary,2790
5,motorcycle,919
6,tram,194
7,train,8794
8,walking,31200
9,metro,1227


In [180]:
overlapping_points_query = """
select min(point.id) as id, latitude, longitude, time from point
join track on track.id = track_id
where point.time between (select min(time) from point where track_id = 2503) 
                     and (select max(time) from point where track_id = 2503)
and (track.type is null or track.type != 'Waypoint')
group by latitude, longitude, time
order by time
"""
display(pd.read_sql_query(overlapping_points_query, engine))

,id,latitude,longitude,time
0,89057,35.710963,139.778258,2019-12-09 22:43:30+00:00
1,91470,35.711317,139.777343,2019-12-09 22:44:43+00:00
2,91471,35.711175,139.777160,2019-12-09 22:44:50+00:00
3,91472,35.711037,139.776997,2019-12-09 22:44:56+00:00
4,91473,35.711046,139.776981,2019-12-09 22:44:58+00:00
...,...,...,...,...
2141,89166,32.867430,-97.054870,2019-12-10 20:02:14+00:00
2142,89167,32.878540,-97.054850,2019-12-10 20:02:30+00:00
2143,89168,32.889270,-97.054760,2019-12-10 20:02:46+00:00
2144,89078,30.299531,-97.713281,2019-12-10 23:41:00+00:00


In [201]:
overlapping_points = [session.query(GPSPoint).get(int(t)) for t in pd.read_sql_query(overlapping_points_query, engine)[['id']].values]

In [202]:
len(overlapping_points)

2130

In [203]:
merged = GPSTrack(name="merged")

In [204]:
merged.points = [clone_model(p, track=merged) for p in overlapping_points]

In [205]:
m2 = Map(center=(0., 0.), zoom=2, close_popup_on_click=False)
m2.add_layer(merged.as_polyline(
        fill=False,
        weight=1))

m2

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [206]:
session.add(merged)

In [207]:
session.commit()

In [133]:
longitudes0 = [p[1] for p in merged.as_polyline().locations[0]]

In [134]:
min(longitudes0), max(longitudes0)

(139.76865116606643, 180)

In [142]:
longitudes1 = [p[1] for p in merged.as_polyline().locations[1]]

In [143]:
min(longitudes1), max(longitudes1)

(-180, -97.05479)

In [193]:
merged.as_polyline().locations[1][-10:]

[[32.78604, -97.05542],
 [32.79781, -97.05531],
 [32.81282, -97.0552],
 [32.81749, -97.0552],
 [32.8319, -97.05507],
 [32.85068, -97.05498],
 [32.86002, -97.05498],
 [32.86958, -97.05493],
 [32.88813, -97.05479],
 [34.82128519222763, -180]]

In [192]:
merged.as_polyline().locations[2]

[[34.82128519222763, 180],
 [35.74459605167583, 140.38382565449308],
 [34.86718898058325, 180]]

In [194]:
query = """
select point.*, track.* from track
join point on track.id = track_id
where latitude = 35.74459605167583
"""
display(pd.read_sql_query(query, engine))

,id,latitude,longitude,elevation,time,tz,properties,track_id,id,name,comment,description,source,type,filename,properties,raw
0,94130,35.744596,140.383826,42.217523,2019-12-10 13:07:54+00:00,None,{},2641,2641,None,None,None,None,None,None,None,None


In [208]:
query = """
select * from track
order by id desc
limit 5
"""
display(pd.read_sql_query(query, engine))

,id,name,comment,description,source,type,filename,properties,raw
0,2644,merged,None,None,None,None,None,None,None
1,2640,NRT - GATE 74 (Terminal 2),None,None,None,Waypoint,None,None,None
2,2639,ASIAN CAFE Bowl Bowl,None,None,None,Waypoint,None,None,None
3,2638,Terminal 2 (第2ターミナル),None,None,None,Waypoint,None,None,None
4,2637,Keisei Narita Airport Terminal 2-3 Station (KS...,None,None,None,Waypoint,None,None,None


In [196]:
track2641 = session.query(GPSTrack).get(2641)

In [200]:
session.delete(session.query(GPSTrack).get(2643))
session.commit()